In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Dropout, LSTM
import numpy as np
from EDA.DataReader import DataReader
from keras.src.legacy.preprocessing.text import Tokenizer
from keras.src.utils import pad_sequences
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

In [ ]:
# dataReader = DataReader("UIT_VFSC") # UIT
dataReader = DataReader("") # tập foody_raw
df_train =dataReader.df_train
df_test = dataReader.df_test
# Tiền xử lý văn bản
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train["corpus"])
max_words = len(tokenizer.word_index) + 1 
sequences = tokenizer.texts_to_sequences(df_train["corpus"])
X_train = pad_sequences(sequences)
y_train = np.array(df_train["label"])
X_Test = pad_sequences(tokenizer.texts_to_sequences(df_test["corpus"]))
y_test = np.array(df_test["label"])
# Xây dựng mô hình LSTM
model = tf.keras.models.Sequential([
    Embedding(input_dim=max_words, output_dim=128),
    LSTM(128, dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    # Dense(3, activation='softmax')  # Multiclass UIT
    Dense(2, activation='softmax') # 2 classes foody
])

# Biên dịch mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=10)
y_pred = model.predict(X_Test)
y_pred_classes = np.argmax(y_pred, axis=1)
acc = accuracy_score(y_test, y_pred_classes)

print(f"Accuracy: {acc:.4f}")

# use for multiclass (UIT)
# mf1 = f1_score(df_test["label"], y_pred_classes, average='macro')
# wf1 = f1_score(df_test["label"], y_pred_classes, average='weighted')

# print(f"F1-Score: {max(mf1, wf1):.4f}")

# use for 2 class (foody)
f1 = f1_score(y_test, y_pred_classes)
print(f"F1 Score: {f1:.4f}")